In [ ]:

import psycopg2

def get_connection():
    return psycopg2.connect(
        dbname="platform", 
        user="postgres", 
        password="password", 
        host="127.0.0.1", 
        port="5432"
    )

In [ ]:
connection = get_connection()
with connection.cursor() as cursor:
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS bank (id SERIAL PRIMARY KEY, account VARCHAR(255), balance FLOAT);
        """
    )

connection.commit()
connection.close()

In [ ]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("INSERT INTO bank (account, balance) VALUES ('Alice', 100);")
    cursor.execute("INSERT INTO bank (account, balance) VALUES ('Bob', 100);")

connection.commit()
connection.close()

In [ ]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("UPDATE bank SET balance = 100 WHERE account = 'Alice';")
    cursor.execute("UPDATE bank SET balance = 100 WHERE account = 'Bob';")


connection.commit()
connection.close()

# Dirty Read:
A transaction reads data that has been modified by another transaction but not yet committed. If the other transaction rolls back, the data read by the first transaction becomes invalid.

# Non-Repeatable Read:
A transaction reads the same data multiple times and gets different values each time because another transaction has modified the data and committed those changes in between the reads.

In [ ]:
connection1 = get_connection()

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("SELECT * FROM bank;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
connection1.commit()
connection1.close()

In [ ]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("UPDATE bank SET balance = 30 WHERE account = 'Alice';")

connection.commit()
connection.close()

# Phantom Read:
A transaction reads a set of rows that satisfy a condition, and another transaction inserts or deletes rows that satisfy the same condition. The first transaction re-executes the query and sees a different set of rows.

In [ ]:
connection1 = get_connection()

In [ ]:
connection1.commit()
connection1.close()

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("SELECT * FROM bank WHERE balance > 50;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("INSERT INTO bank (account, balance) VALUES ('Dan', 100);")

connection.commit()
connection.close()

# Lost Update:
Two or more transactions read the same data and then update it based on the read value. The final update will overwrite the changes made by the other transactions, leading to lost updates.

In [ ]:
connection1 = get_connection()
connection2 = get_connection()


In [ ]:
connection1.commit()
connection1.close()

In [ ]:
connection2.commit()
connection2.close()

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("SELECT * FROM bank;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
with connection2.cursor() as cursor:
    cursor.execute("SELECT * FROM bank;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("UPDATE bank SET balance = balance - 50 WHERE account = 'Alice';")
    cursor.execute("UPDATE bank SET balance = balance + 50 WHERE account = 'Bob';")
    

In [ ]:
with connection2.cursor() as cursor:
    cursor.execute("UPDATE bank SET balance = balance + 100 WHERE account = 'Alice';")
    cursor.execute("UPDATE bank SET balance = balance - 100 WHERE account = 'Bob';")

# Write Skew:
Two transactions read overlapping sets of data and then update non-overlapping sets of data. The final state of the database may violate integrity constraints because the transactions were unaware of each other's updates.

In [ ]:
connection = get_connection()
with connection.cursor() as cursor:
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS hospital (id SERIAL PRIMARY KEY, duty Boolean);
        """
    )
    cursor.execute("INSERT INTO hospital (duty) VALUES (True);")
    cursor.execute("INSERT INTO hospital (duty) VALUES (True);")
        
connection.commit()
connection.close()

In [ ]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM hospital;")
    for row in cursor.fetchall():
        print(row)

connection.commit()
connection.close()

In [ ]:
connection1 = get_connection()
connection2 = get_connection()

In [ ]:
connection1.commit()
connection1.close()

In [ ]:
connection2.commit()
connection2.close()

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("SELECT * FROM hospital;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
with connection2.cursor() as cursor:
    cursor.execute("SELECT * FROM hospital;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("UPDATE hospital SET duty = false WHERE id = 1;")

In [ ]:
with connection2.cursor() as cursor:
    cursor.execute("UPDATE hospital SET duty = false WHERE id = 2;")